In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import pairwise_kernels
import scipy
from sklearn import svm, linear_model
from sklearn.model_selection import GridSearchCV
import time
import pandas as pd
from sklearn.model_selection import KFold

In [2]:
nameFile = open("features.txt")
nameList = []
for feature in nameFile:
    feature = feature.strip()
    nameList.append(feature)
X_train = pd.read_csv("cleaned_X_train.csv", names = nameList)
X_test = pd.read_csv("cleaned_X_test.csv", names = nameList)


y_train = pd.read_csv("y_train.csv", names = ["Action"])
y_test = pd.read_csv("y_test.csv", names = ["Action"])

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

train_sub = open('subject_id_train.txt')
test_sub = open('subject_id_test.txt')
train_id_dict = {}
test_id_dict = {}

for line in train_sub:
    line = int(line.strip())

    if line in train_id_dict:
        train_id_dict[line] += 1
    else:
        train_id_dict[line] = 1
        
for line in test_sub:
    line = int(line.strip())

    if line in test_id_dict:
        test_id_dict[line] += 1
    else:
        test_id_dict[line] = 1


start = 0
for key in train_id_dict:
    temp = train_id_dict[key]
    temp += start
    train_id_dict[key] = (start, temp - 1)
    start = temp

for key in test_id_dict:
    temp = test_id_dict[key]
    temp += start
    test_id_dict[key] = (start, temp - 1)
    start = temp
    
print(train_id_dict)

{1: (0, 367), 3: (368, 726), 5: (727, 1050), 6: (1051, 1392), 7: (1393, 1710), 8: (1711, 2005), 11: (2006, 2342), 14: (2343, 2688), 15: (2689, 3039), 16: (3040, 3418), 17: (3419, 3819), 19: (3820, 4200), 21: (4201, 4634), 22: (4635, 4969), 23: (4970, 5361), 25: (5362, 5795), 26: (5796, 6204), 27: (6205, 6598), 28: (6599, 6993), 29: (6994, 7357), 30: (7358, 7766)}


In [3]:
outFile = open("per_participant_training_results.txt", 'w')
for subject in train_id_dict:
    sub_X = X_train[train_id_dict[subject][0]:train_id_dict[subject][1]+1]
    sub_y = y_train[train_id_dict[subject][0]:train_id_dict[subject][1]+1]
    
    print(sub_X.shape)
    print(sub_y.shape)
    kf = KFold(n_splits=5)
    
    #Why does this only work for the first subject only? For the second subject, it seems like we are unable to slice 
    #X_train using [start:end] because the slice always turns out to be 0 rows. The start and end were defined correctly
    #as displayed in the print statement, but the slicing just does not work.
    
    accuracy = 0
    
    for train_index, test_index in kf.split(sub_X):

        sub_X_train, sub_X_test = sub_X[train_index[0]:train_index[-1]+1], sub_X[test_index[0]:test_index[-1]+1]
        sub_y_train, sub_y_test = sub_y[train_index[0]:train_index[-1]+1], sub_y[test_index[0]:test_index[-1]+1]

        svc = svm.SVC(probability=False,  kernel="rbf", C=10, gamma=.05,verbose=10)
        svc.fit(sub_X_train, sub_y_train)

        yhat_ts = svc.predict(sub_X_test)





        acc = np.mean(yhat_ts == sub_y_test)
        accuracy += acc
        print('Accuaracy for subject {0} = {1:f}'.format(subject, acc), file = outFile)
        
    print('Average accuaracy for subject {0} = {1:f}'.format(subject, accuracy/5), file = outFile)
outFile.close()

(368, 561)
(368,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](359, 561)
(359,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](324, 561)
(324,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](342, 561)
(342,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](318, 561)
(318,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](295, 561)
(295,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](337, 561)
(337,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](346, 561)
(346,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](351, 561)
(351,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](379, 561)
(379,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](401, 561)
(401,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](381, 561)
(381,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](434, 561)
(434,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](335, 561)
(335,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](392, 561)
(392,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](434, 561)
(434,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](409, 561)
(409,)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM](394, 561)
(39